In [1]:
#testing connection to SQL Server database using pyodbc
import pyodbc as odbc


#connection string components
server = "localhost"
database = "mydb"
username = "sa"
password = r"N\VHs8*DJV"
driver = "/opt/homebrew/lib/libmsodbcsql.18.dylib"



#connection string. Using trustcertificate=yes because I'm using a self-signed certificate
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes"

#making the connection using a try-except block
try:
    conn = odbc.connect(connection_string)
    print("Connected successfully!")

except odbc.Error as ex:
    print("Connection failed:", ex)

conn.close()

[ODBC][13368][1700818062.511813][__handles.c][499]
		Exit:[SQL_SUCCESS]
			Environment = 0x13685b200
[ODBC][13368][1700818062.511846][SQLSetEnvAttr.c][189]
		Entry:
			Environment = 0x13685b200
			Attribute = SQL_ATTR_ODBC_VERSION
			Value = 0x3
			StrLen = 4
[ODBC][13368][1700818062.511867][SQLSetEnvAttr.c][381]
		Exit:[SQL_SUCCESS]
[ODBC][13368][1700818062.511892][SQLAllocHandle.c][395]
		Entry:
			Handle Type = 2
			Input Handle = 0x13685b200
		UNICODE Using encoding ASCII 'UTF-8' and UNICODE 'UCS-2-INTERNAL'

[ODBC][13368][1700818062.511942][SQLAllocHandle.c][531]
		Exit:[SQL_SUCCESS]
			Output Handle = 0x13684d000
[ODBC][13368][1700818062.514311][SQLDriverConnectW.c][298]
		Entry:
			Connection = 0x13684d000
			Window Hdl = 0x0
			Str In = [DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost;DATABASE=mydb;UID=sa;PWD=**********;TrustServerCertificate=yes][length = 126 (SQL_NTS)]
			Str Out = 0x0
			Str Out Max = 0
			Str Out Ptr = 0x0
			Completion = 0
[ODBC][13368][170

NameError: name 'conn' is not defined

In [ ]:
#this is an example of how to use the connection object to execute a query

#creating a cursor object from the connection object, because the cursor object has the execute method
#cursor = conn.cursor()

# executing a query that gets some of the data from the sample table
#cursor.execute("SELECT CPR, SampleID, SampleDate FROM mydb.test_api_schema.Sample_data")

#to get the results from the query, we use the fetchall method on the cursor object. This returns a list of tuples
#result = cursor.fetchall()
#if result:
#    print(result)

# we then close the connection, because we don't need it anymore and it's good practice
#cursor.close()
#conn.close()

In [ ]:
# my password
#import os
# in bash: export DATABASE_PASSWORD="N\VHs8*DJV"



In [6]:
# inserting xxxx.json file into the database (remember to specify)

#getting my password from my environment file
import os
from dotenv import load_dotenv
# this loads the config.env file when the app is opened
load_dotenv()

# importing json and pyodbc
import pyodbc as odbc
import json

# connection to the database, by creating a function that takes the connection string as a parameter
def establish_db_connection(connection_string):
    """Establish a database connection using the provided connection string.
    
    Returns:
        conn (object): The connection object for the database.
        cursor (object): A cursor object to interact with the database. Points to specific rows in tables.
    """
    try:
        conn = odbc.connect(connection_string)
        cursor = conn.cursor()
        return conn, cursor
    except odbc.Error as ex:
        print("Connection failed:", ex)
        return None, None

# Function that reads a json file, where the parameter it takes is a file path that points to a json file
def read_json_file(file_path):
    """
    Reads the content of a JSON file and returns it as a Python object.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        list: A Python object representing the JSON structure, which is a list of dictionaries.
                Returns None if an error occurs during reading.

    Raises:
        Exception: If there's an issue reading the file, an exception is raised and handled within the function.
                   The specific error message is printed to the console.
    """
    try:
        with open(file_path, "r") as json_file:
            data = json.load(json_file)
            return data
    except Exception as e:
        print(f"error reading the file {e}")
        return None

# function that takes 3 parameteres, the connection object, cursor object and the python object exstracted from the json file
def insert_into_database(conn, cursor, data, table_name, columns):
    """ 
    Inserts a list of data into the specified database table.
    
    The function expects data to be a list of dictionaries where each dictionary 
    corresponds to a record to be inserted into the database.

    Args:
        conn (object): The connection object for the database.
        cursor (object): A cursor object to interact with the database.
        data (list): A list of dictionaries, each representing a record to be inserted.
        table_name (str): The name of the table into which data will be inserted.
        columns (list): List of columns names in which data will be inserted.

    Returns:
        None. However, it commits changes to the database and prints a confirmation message upon successful insertion.
        
    Raises:
        Exception: If there is an issue inserting the data into the database, an exception will be raised and printed.
    """
    # we define the value placeholders for the rows of the columns, this is for security and to avoid any SQL injections (massive SQL insertions from outside our control)
    placeholders = ', '.join('?' for col in columns)
    # we join the column names together, separated by a comma and a space --> specified when calling the function ["CPR", "Phone_number", "Region", "Gender"]
    column_names = ', '.join(columns)
    # we define the actual SQL query. (we set VALUES to insert as "?")
    query = f"INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})"
    
    try:
        # Each "row" in the data list is a dictionary representing a record (1 CPR, 1 Phone_number, 1 Region, 1 Gender).
        for row in data:
        # For each "row", we extract values corresponding to the column names defined in the "columns" list and save them as a tuple.
            values = tuple(row[col] for col in columns)
        # We use the execute method on the cursor object to run the SQL query, passing in the tuple of values.
            cursor.execute(query, values)
        
        print("The data has been inserted")
        # we then use the commit method on the conn object, to push the query through to the database.
        conn.commit()
    except Exception as e:
        print("something went wrong", e)
        
    print(os.getenv('DATABASE_PASSWORD'))
#creating main function, because we want to be able to import this file without running the code
def main(json_file_name, table_name, columns):
    """
    Reads data from a specified JSON file and inserts it into the specified table in a database.
    
    Args:
        json_file_name (str): The path to the JSON file containing data to be inserted.
        table_name (str): The fully qualified name (i.e., schema.table) of the table into which the data should be inserted.
        columns (list): A list of strings representing the column names in the order they appear in the JSON file's dictionaries.

    Notes:
        Database connection details are hardcoded within the function. In production, consider moving these settings 
        to a more secure location such as a configuration file or environment variables. (This is just a test for how it might look)

    Returns:
        None. However, function prints status messages to indicate progress or errors.
        
    Raises:
        Exception: If there is an issue during the process, an exception will be raised and printed.
    """
    server = "localhost"
    database = "mydb"
    username = "sa"
    password = 'Slotved2314!' # getting the password from the config.env file
    # direct path: "/opt/homebrew/lib/libmsodbcsql.18.dylib"
    driver = "{ODBC Driver 18 for SQL Server}"
    connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes"
    
    conn, cursor = establish_db_connection(connection_string)
    if cursor:
        try:
            data = read_json_file(json_file_name)
            if data:
                insert_into_database(conn, cursor, data, table_name, columns)
                print("Data has been inserted")
            else:
                print("data file is empty")
        except Exception as e:
            print(f'an error has occurred {e}')    
        finally:
            cursor.close()
            conn.close()

    
if __name__ == "__main__":
    
    # main call, make sure to adapt to your needs
    # see args in doc string for function call
    
    main("Persons.json", "QC.Persons", ["CPR", "Phone_number", "Region", "Gender"])
    main("Sample.json", "QC.Sample", ["SampleID", "CPR", "SampleDate", "Host", "Ct"])
    main("Batch.json", "QC.Batch", ["BatchID", "BatchDate", "Platform", "BatchSource"])
    main("SequencedSample.json", "QC.SequencedSample", ["SequencedSampleID", "SampleContent", "DateSequencing", "Quality", "Organism", "OrganismID", "SampleID", "BatchID"])
    main("Covid.json", "QC.COVID19", ["CovidID", "Pango_designation", "WhoVariant", "QcScore"])
    main("Legionella.json", "QC.Legionella", ["LegionellaID", "Genotype", "Disease", "DiseasePhenotype", "DanishLocation", "ForeignLocation", "AcquiredFood"])
    main("Saureus.json", "QC.s_aureus", ["AreusID", "Genotype", "Disease", "DiseasePhenotype", "DanishLocation", "ForeignLocation", "AcquiredHospital", "AcquiredSurgery", "Infectionlocation"])
    main("Epidermidis.json", "QC.S_epidermidis", ["EpidermidisID", "Genotype", "Disease", "DiseasePhenotype", "DanishLocation", "ForeignLocation", "AcquiredHospital", "AcquiredSurgery", "Infectionlocation"])
    

    #deleting files, if they are successfully inserted into the database
    
        


[ODBC][13368][1700819564.287380][SQLAllocHandle.c][395]
		Entry:
			Handle Type = 2
			Input Handle = 0x13685b200
		UNICODE Using encoding ASCII 'UTF-8' and UNICODE 'UCS-2-INTERNAL'

[ODBC][13368][1700819564.287611][SQLAllocHandle.c][531]
		Exit:[SQL_SUCCESS]
			Output Handle = 0x146953c00
[ODBC][13368][1700819564.287816][SQLDriverConnectW.c][298]
		Entry:
			Connection = 0x146953c00
			Window Hdl = 0x0
			Str In = [DRIVER={ODBC Driver 18 for SQL Server};SERVER=localhost;DATABASE=mydb;UID=sa;PWD=************;TrustServerCertificate=yes][length = 120 (SQL_NTS)]
			Str Out = 0x0
			Str Out Max = 0
			Str Out Ptr = 0x0
			Completion = 0
[ODBC][13368][1700819564.303583][SQLConnect.c][1145]Can't open lib 'ODBC Driver 18 for SQL Server' : file not found
[ODBC][13368][1700819564.303647][SQLDriverConnect.c][751]
		Entry:
			Connection = 0x146953c00
			Window Hdl = 0x0
			Str In = [DRIVER={ODBC Driver 18 for SQL Server};SERVER=localhost;DATABASE=mydb;UID=sa;PWD=************;TrustServerCertificat


			Param Type = 1
			C Type = -8 SQL_C_WCHAR
			SQL Type = -9 SQL_WVARCHAR
			Col Def = 12
			Scale = 0
			Rgb Value = 0x10839a0f0
			Value Max = 12
			StrLen Or Ind = 0x1167b5cc0
[ODBC][13368][1700819565.274025][SQLBindParameter.c][434]
		Exit:[SQL_SUCCESS]
[ODBC][13368][1700819565.274034][SQLBindParameter.c][217]
		Entry:
			Statement = 0x120025800
			Param Number = 7
			Param Type = 1
			C Type = -8 SQL_C_WCHAR
			SQL Type = -9 SQL_WVARCHAR
			Col Def = 8
			Scale = 0
			Rgb Value = 0x1083994f0
			Value Max = 8
			StrLen Or Ind = 0x1167b5d28
[ODBC][13368][1700819565.274043][SQLBindParameter.c][434]
		Exit:[SQL_SUCCESS]
[ODBC][13368][1700819565.274051][SQLBindParameter.c][217]
		Entry:
			Statement = 0x120025800
			Param Number = 8
			Param Type = 1
			C Type = -8 SQL_C_WCHAR
			SQL Type = -9 SQL_WVARCHAR
			Col Def = 2
			Scale = 0
			Rgb Value = 0x108399580
			Value Max = 2
			StrLen Or Ind = 0x1167b5d90
[ODBC][13368][1700819565.274060][SQLBindParameter.c][434]
		Exit:[SQL_SUCCESS

In [ ]:
#deleting files
import os # os module is used to delete files
file_paths = ["Persons.json", "Sample.json", "Batch.json", "SequencedSample.json", "Batch_data.csv", "Persons_data.csv", "Sample_data.csv", 
              "SequencedSample_data.csv", "Covid_data.csv", "Covid.json", "Legionella_data.csv", "Legionella.json", "Saureus_data.csv",
              "Saureus.json", "Epidermidis_data.csv", "Epidermidis.json"]
    
def delete_files(file_paths):
    for file_path in file_paths:
        try:
            os.remove(file_path)
        except FileNotFoundError:
            print(f"file {file_path} not found")
            continue
        except Exception as e:
            print(f"error deleting file {e}")

delete_files(file_paths)